This code will go through the groundtruth clip videos on the server and randomly choose 3 non-overlapping clips from each track to score. It will extract the observation, track, start frame and endframe from the file name and store these in a dataframe. It will then play the video and allow the user to input a behavioral score for the clip. To score the video, the user will choose one of three categories (encamped, walking, running) that describes the whole clip. If the clip shows more than one category, it will be discarded and another clip will be chosen. 

The information about the video will be saved along with the behavioral score in a dataframe.

In [7]:
# import packages
import pandas as pd
import numpy as np
import os
import random
import cv2
import glob
import shutil

mount = '/home/blair/server/herd_hover'
video_folder = os.path.join(mount, 'vigilance', 'groundtruth-clips')

In [8]:
# read in dataframe of previously scored videos

# # If no videos have been scored yet:
# d = {'obtrack': [], 'start': [], 'end': [], 'score': []}
# scored = pd.DataFrame(d)

# If some videos have already been scored:
scored = pd.read_csv('video_scores.csv')

In [9]:
# get all filenames in the video folder
vids = []
for root, dirs, files in os.walk(video_folder):
    for name in files:
        if name.startswith('.'):
            continue
        vids.append(name)

In [10]:
obtracks = []
startframe = []
endframe = []
for v in vids:
    ob = v.split('_')[0]
    tr = v.split('_')[1]
    obtr = str(ob + '-' + tr)
    start = int(v.split('_')[3].split('.')[0])
    end = start + 300
    obtracks.append(obtr)
    startframe.append(start)
    endframe.append(end)

In [11]:
d = {'obtrack':obtracks, 'start':startframe, 'end':endframe, 'file': vids}
allvids = pd.DataFrame(d)

In [12]:
for i in np.arange(len(allvids)):
    tr = allvids.obtrack[i]
    # If we have already scored 3 videos for this track, move on 
    if len(scored[scored.obtrack == tr]) >2:
        continue
    # If we have already scored one or two videos for this track, we need to make sure the current video does not
    # overlap those videos
    if len(scored[scored.obtrack == tr]) >0:
        check = scored[scored.obtrack == tr]
        skip = False
        for c in check.start:
            if (allvids.start[i] > c) & (allvids.start[i] < c + 300):
                skip = True
        if skip == True:
            continue
    print('watch and score video ' + allvids.file[i])
    print('type stationary, walking or running, or split')
    score = input()
    video = os.path.join(video_folder, allvids.file[i])
    if score == 'split':
        new_loc = os.path.join(video_folder, 'scored', 'split', allvids.file[i])
        shutil.move(video, new_loc)
        continue
    if score == 'skip':
        continue
    tr = allvids.obtrack[i]
    start = allvids.start[i]
    end = allvids.end[i]
    new_row = {'obtrack': tr, 'start': start, 'end': end, 'score': score}
    new_row = pd.DataFrame(new_row, index = [0])
    scored = pd.concat([scored, new_row], ignore_index = True)
    new_loc = os.path.join(video_folder, 'scored', allvids.file[i])
    shutil.move(video, new_loc)

watch and score video observation027_track-6_startframe_04860.mp4
type stationary, walking or running, or split


 running


watch and score video observation027_track-0_startframe_01740.mp4
type stationary, walking or running, or split


 running


watch and score video observation027_track-3_startframe_08520.mp4
type stationary, walking or running, or split


...

watch and score video observation066_track-14_startframe_64500.mp4
type stationary, walking or running, or split


 running


watch and score video observation066_track-14_startframe_64500.mp4
type stationary, walking or running, or split


 running


FileNotFoundError: [Errno 2] No such file or directory: '/home/blair/server/herd_hover/vigilance/groundtruth-clips/observation066_track-14_startframe_64500.mp4'

In [13]:
scored.drop_duplicates(inplace = True)
scored.to_csv('video_scores.csv', index = False)

In [14]:
scored.score.value_counts()

stationary    82
walking       29
running       17
Name: score, dtype: int64